In [ ]:
import json, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from aquabyte.data_access_utils import S3AccessUtils, RDSAccessUtils

In [ ]:
#rds_access_utils = RDSAccessUtils(json.load(open(os.environ['DATA_WAREHOUSE_SQL_CREDENTIALS'])))
rds_access_utils = RDSAccessUtils(json.load(open(os.environ['PROD_SQL_CREDENTIALS'])))
                                                    
query = """
    select * from sites;
"""

df_sites = rds_access_utils.extract_from_database(query)

df_sites

In [ ]:
query = """
    select captured_at, annotation, annotation_metadata 
    from annotations a
    where a.pen_id = 56
    and a.is_qa = true
    and a.is_skipped = false
    and a.captured_at > '2020-03-01';
"""

lice_counts = rds_access_utils.extract_from_database(query)

lice_counts

In [ ]:
counts = {
    'NUM_FISH': {
        'LEFT': 0,
        'RIGHT': 0 
    },
    'ADULT_FEMALE': {
        'LEFT': 0,
        'RIGHT': 0
    },
    'MOVING': {
        'LEFT': 0,
        'RIGHT': 0
    }
}

categories = ['ADULT_FEMALE', 'MOVING']
directions = ['LEFT', 'RIGHT', 'TOP', 'BOTTOM']

for direction in directions:
    lice_counts[direction] = 0 
    
    for category in categories:
        categoryDirection = category + '_' + direction
        
        lice_counts[categoryDirection] = 0 


for index, lice_count in lice_counts.iterrows():   
    direction = lice_count['annotation_metadata']['direction']
    
    lice_counts.ix[index, 'movingCountAdjusted'] = lice_count['annotation_metadata']['liceCounts']['movingCountAdjusted']
    lice_counts.ix[index, 'adultFemaleCountAdjusted'] = lice_count['annotation_metadata']['liceCounts']['adultFemaleCountAdjusted']
        
    leftAnnotations = lice_count['annotation']['leftCrop'] or []
    rightAnnotations = lice_count['annotation']['rightCrop'] or []
    annotations = leftAnnotations + rightAnnotations
    
    lice_counts.ix[index, direction] = lice_counts.ix[index, direction] + 1
    counts['NUM_FISH'][direction] = counts['NUM_FISH'][direction] + 1
    
    for annotation in annotations:
        category = annotation['category']
        liceLocation = annotation['liceLocation']
        
        categoryDirection = category + '_' + direction
        categoryDirectionLocation = category + '_' +  liceLocation
        
        if liceLocation == 'MIDDLE':
            lice_counts.ix[index, categoryDirection] = lice_counts.ix[index, categoryDirection] + 1
            counts[category][direction] = counts[category][direction] + 1
        else:
            lice_counts.ix[index, categoryDirectionLocation] = lice_counts.ix[index, categoryDirection] + 1

In [ ]:
counts

In [ ]:
directions = ['LEFT', 'RIGHT']

for direction in directions:
    af_ratio = counts['ADULT_FEMALE'][direction] / counts['NUM_FISH'][direction]
    m_ratio = counts['MOVING'][direction] / counts['NUM_FISH'][direction]
    print('%s: AF: %0.3f, M: %0.3f' % (direction, af_ratio, m_ratio))

In [ ]:
lice_counts

In [ ]:
columns = [ 'ADULT_FEMALE_LEFT', 'ADULT_FEMALE_RIGHT', 'MOVING_LEFT', 'MOVING_RIGHT', 'LEFT', 'RIGHT', 'ADULT_FEMALE_TOP', 'ADULT_FEMALE_BOTTOM', 'MOVING_TOP', 'MOVING_BOTTOM', 'adultFemaleCountAdjusted', 'movingCountAdjusted', ]
left_columns = [ 'ADULT_FEMALE_LEFT', 'MOVING_LEFT' ]
right_columns = [ 'ADULT_FEMALE_RIGHT', 'MOVING_RIGHT' ]

lice_counts.index = pd.to_datetime(lice_counts.captured_at)

df_daily = lice_counts[columns].rolling('D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
df_daily_left = lice_counts.loc[lice_counts['LEFT'] == 1, left_columns].rolling('D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)
df_daily_right = lice_counts.loc[lice_counts['RIGHT'] == 1, right_columns].rolling('D').mean().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)

In [ ]:
df_daily[left_columns] = df_daily_left[left_columns]
df_daily[right_columns] = df_daily_right[right_columns]
df_daily

In [ ]:
#np.sum(df_daily['MOVING_LEFT']) / np.sum(df_daily['LEFT'])

np.sum(df_daily['MOVING_RIGHT']) / np.sum(df_daily['RIGHT'])


In [ ]:
df_daily = df_daily.dropna()

In [ ]:
df_daily

In [ ]:
df_num = lice_counts[['LEFT', 'RIGHT']].rolling('D').sum().resample('D').apply(lambda x:x.tail(1) if x.shape[0] else np.nan)

In [ ]:
df_daily['NUM_FISH'] = df_num['LEFT'] + df_num['RIGHT']

In [ ]:
df_daily

In [ ]:
fig, ax = plt.subplots(2)

fig.set_size_inches(10, 20)

af_count = df_daily['ADULT_FEMALE_LEFT'] + df_daily['ADULT_FEMALE_RIGHT'] + df_daily['ADULT_FEMALE_TOP'] + df_daily['ADULT_FEMALE_BOTTOM']
m_count = df_daily['MOVING_LEFT'] + df_daily['MOVING_RIGHT'] + df_daily['MOVING_TOP'] + df_daily['MOVING_BOTTOM']

ax[0].plot(df_daily.index, df_daily['ADULT_FEMALE_LEFT'], linestyle = '-', marker = 'o', color = 'red', label = 'Left AF')
ax[0].plot(df_daily.index, df_daily['ADULT_FEMALE_RIGHT'], linestyle = '-', marker = 'o', color = 'blue', label = 'Right AF')
ax[0].plot(df_daily.index, df_daily['LEFT'], linestyle = '--', marker = 'o', color = 'green', label = 'Pct Left')
ax[0].plot(df_daily.index, af_count, linestyle = '--', marker = 'o', color = 'black', label = 'Avg AF')
ax[0].plot(df_daily.index, df_daily['adultFemaleCountAdjusted'], linestyle = '--', marker = '*', color = 'purple', label = 'Avg AF')
ax[0].set_title('Vikane Pen 1 Adult Female Counts by Side')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Count')

ax[1].plot(df_daily.index, df_daily['MOVING_LEFT'], linestyle = '-', marker = 'o', color = 'red', label = 'Left M')
ax[1].plot(df_daily.index, df_daily['MOVING_RIGHT'], linestyle = '-', marker = 'o', color = 'blue', label = 'Right M')
ax[1].plot(df_daily.index, df_daily['LEFT'], linestyle = '--', marker = 'o', color = 'green', label = 'Pct Left')
ax[1].plot(df_daily.index, m_count, linestyle = '--', marker = 'o', color = 'black', label = 'Avg M')
ax[1].plot(df_daily.index, df_daily['movingCountAdjusted'], linestyle = '--', marker = '*', color = 'purple', label = 'Avg AF')
ax[1].set_title('Vikane Pen 1 Mobile Counts by Side')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Count')

ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.show()

In [ ]:
for index, lice_count in df_daily.iterrows():
    df_daily.ix[index, 'leftAFCount'] = lice_count['ADULT_FEMALE_LEFT'] * lice_count['NUM_FISH']
    df_daily.ix[index, 'rightAFCount'] = lice_count['ADULT_FEMALE_RIGHT'] * lice_count['NUM_FISH']
    df_daily.ix[index, 'leftMCount'] = lice_count['MOVING_LEFT'] * lice_count['NUM_FISH']
    df_daily.ix[index, 'rightMCount'] = lice_count['MOVING_RIGHT'] * lice_count['NUM_FISH']

leftAFCount = []
rightAFCount = []
leftMCount = []
rightMCount = []
    
for index, lice_count in df_daily.iterrows():
    if len(leftCount) == 7:
        
        df_daily.ix[index, 'LRCoefficientAF'] = np.sum(leftAFCount) / np.sum(rightAFCount)
        df_daily.ix[index, 'LRCoefficientM'] = np.sum(leftMCount) / np.sum(rightMCount)

    leftAFCount.append(lice_count['leftAFCount'])
    rightAFCount.append(lice_count['rightAFCount'])
    leftMCount.append(lice_count['leftMCount'])
    rightMCount.append(lice_count['rightMCount'])
    
    if len(leftAFCount) > 7:
        leftAFCount.pop(0)
        rightAFCount.pop(0)
        leftMCount.pop(0)
        rightMCount.pop(0)
        
df_daily

In [ ]:
plt.plot(df_daily.index, df_daily['LRCoefficientAF'], linestyle = '--', marker = 'o', color = 'red')
plt.plot(df_daily.index, df_daily['LRCoefficientM'], linestyle = '--', marker = 'o', color = 'blue')
plt.show()

In [ ]:
fig, ax = plt.subplots(2)

fig.set_size_inches(10, 20)

df_daily = df_daily.dropna()

af_count = df_daily['ADULT_FEMALE_LEFT'] + df_daily['ADULT_FEMALE_RIGHT'] + df_daily['ADULT_FEMALE_TOP'] + df_daily['ADULT_FEMALE_BOTTOM']
m_count = df_daily['MOVING_LEFT'] + df_daily['MOVING_RIGHT'] + df_daily['MOVING_TOP'] + df_daily['MOVING_BOTTOM']

af_count_left = df_daily['ADULT_FEMALE_LEFT'] * df_daily['LEFT'] + (1 - df_daily['LEFT']) * df_daily['ADULT_FEMALE_RIGHT'] * df_daily['LRCoefficientAF']
af_count_right = df_daily['ADULT_FEMALE_RIGHT'] * df_daily['RIGHT'] + (1 - df_daily['RIGHT']) * df_daily['ADULT_FEMALE_LEFT'] * (1 / df_daily['LRCoefficientAF'])
af_count_lr = af_count_left + af_count_right + df_daily['ADULT_FEMALE_TOP'] + df_daily['ADULT_FEMALE_BOTTOM']

m_count_left = df_daily['MOVING_LEFT'] * df_daily['LEFT'] + (1 - df_daily['LEFT']) * df_daily['MOVING_RIGHT'] * df_daily['LRCoefficientM']
m_count_right = df_daily['MOVING_RIGHT'] * df_daily['RIGHT'] + (1 - df_daily['RIGHT']) * df_daily['MOVING_LEFT'] * (1 / df_daily['LRCoefficientM'])
m_count_lr = m_count_left + m_count_right + df_daily['MOVING_TOP'] + df_daily['MOVING_BOTTOM']

ax[0].plot(df_daily.index, af_count, linestyle = '-', marker = 'o', color = 'red', label = 'Original')
ax[0].plot(df_daily.index, af_count_lr, linestyle = '--', marker = 'o', color = 'blue', label = 'With Sides')
ax[0].plot(df_daily.index, df_daily['adultFemaleCountAdjusted'], linestyle = '--', marker = '*', color = 'purple', label = 'Avg AF')
ax[0].set_title('Vikane Pen 1 Adult Female Counts by Side')
ax[0].set_xlabel('Date')
ax[0].set_ylabel('Count')

ax[1].plot(df_daily.index, m_count, linestyle = '-', marker = 'o', color = 'red', label = 'Original')
ax[1].plot(df_daily.index, m_count_lr, linestyle = '--', marker = 'o', color = 'blue', label = 'With Sides')
ax[1].plot(df_daily.index, df_daily['movingCountAdjusted'], linestyle = '--', marker = '*', color = 'purple', label = 'Avg AF')
ax[1].set_title('Vikane Pen 1 Mobile Counts by Side')
ax[1].set_xlabel('Date')
ax[1].set_ylabel('Count')

ax[0].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
ax[1].legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

plt.show()